In [ ]:
import sys
import os
import pandas as pd
import numpy as np

from github import Github, Repository
from GithubCredentials import my_credentials
from enum import Enum

sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'harpcrawler'))
from harprepo.device import DeviceRepo, TemplateDeviceRepo
from harprepo.peripheral import PeripheralRepo, TemplatePeripheralRepo

import fileparser


In [ ]:
## Setup
gh = Github(login_or_token=my_credentials())


In [ ]:
## Grab the template repositories
device_template = TemplateDeviceRepo(gh.get_repo("bruno-f-cruz/device.template"))
peripheral_template = TemplatePeripheralRepo(gh.get_repo("bruno-f-cruz/peripheral.template"))

In [ ]:
harp_organization = gh.get_organization("harp-tech")

## Get all "Device.*" repositories
device_repos = [DeviceRepo(repo, device_template)\
    for repo in harp_organization.get_repos()\
        if (("device." in repo.full_name) and not("template" in repo.full_name))]

device_template.run_diagnosis(repos_to_validate=device_repos)
device_diagnosis = device_template.print_diagnosis()
print(device_diagnosis)





if False:
    ## Write to google sheets:
    import gspread
    gc = gspread.service_account()
    sh = gc.open("GitHarpCrawler")
    try:
        worksheet = sh.add_worksheet(title="devices", rows = 100, cols= 100)
    except:
        sh.del_worksheet(sh.worksheet(title="devices"))
        worksheet = sh.add_worksheet(title="devices", rows = 100, cols= 100)
    worksheet.update([device_diagnosis.columns.values.tolist()] + device_diagnosis.values.tolist())



In [ ]:
harp_organization = gh.get_organization("harp-tech")

## Get all "Peripherals.*" repositories

peripheral_repos = [PeripheralRepo(repo, peripheral_template)\
    for repo in harp_organization.get_repos()\
        if (("peripheral." in repo.full_name) and not("template" in repo.full_name))]

peripheral_template.run_diagnosis(repos_to_validate=peripheral_repos)
peripherals_diagnosis = peripheral_template.print_diagnosis()
print(peripherals_diagnosis)


if False:
    ## Write to google sheets:
    import gspread
    gc = gspread.service_account()
    sh = gc.open("GitHarpCrawler")
    try:
        worksheet = sh.add_worksheet(title="peripherals", rows = 100, cols= 100)
    except:
        sh.del_worksheet(sh.worksheet(title="peripherals"))
        worksheet = sh.add_worksheet(title="peripherals", rows = 100, cols= 100)
    worksheet.update([peripherals_diagnosis.columns.values.tolist()] + peripherals_diagnosis.values.tolist())

In [ ]:
for repo in device_repos:
    warnings = fileparser.validate_content(
        repository=repo,
        template_repository=device_template,
        template_files=["README.md"])
    print(f"Repo {repo.repository.name} with {warnings}")